# Data Cleaning

### Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
player_info = pd.read_csv('./Data/nfl_draft_prospects.csv')
draft = pd.read_csv('./Data/nfl_draft_profiles_update.csv')
draft2022 = pd.read_excel('./Data/reports2022.xlsx',index_col=0)

In [3]:
draft2022

,player_name_x,draft_year,pick,text1
6034,Travon Walker,2022,1,Walker is an EDGE defender with an excellent c...
5878,Aidan Hutchinson,2022,2,Hutchinson is a notch below elite in terms of ...
6002,Derek Stingley,2022,3,Stingley is a physical press corner with great...
5846,Sauce Gardner,2022,4,Gardner is the best press-man cornerback in th...
6013,Kayvon Thibodeaux,2022,5,Thibodeaux has the explosive first step to get...
...,...,...,...,...
5858,Jeff Gunter,2022,252,Gunter is a versatile edge defender who lines ...
5849,Trenton Gill,2022,255,Gill has a strong leg. He led the ACC in punt ...
5918,Jesse Luketa,2022,256,Luketa is a relentless run-defender who has go...
5870,Marquis Hayes,2022,257,Hayes is a mauler who plays with good leverage...


In [4]:
scout2 = draft.merge(player_info,how='left',on='player_id')

In [5]:
scout2.columns

Index(['player_id', 'guid_x', 'alt_player_id', 'player_name_x', 'position_x',
       'pos_abbr_x', 'weight_x', 'height_x', 'player_image_x', 'link_x',
       'school_logo', 'school_x', 'school_abbr_x', 'school_name_x', 'pos_rk_x',
       'ovr_rk_x', 'grade_x', 'text1', 'text2', 'text3', 'text4', 'draft_year',
       'player_name_y', 'position_y', 'pos_abbr_y', 'school_y',
       'school_name_y', 'school_abbr_y', 'link_y', 'pick', 'overall', 'round',
       'traded', 'trade_note', 'team', 'team_abbr', 'team_logo_espn', 'guid_y',
       'weight_y', 'height_y', 'pos_rk_y', 'ovr_rk_y', 'grade_y',
       'player_image_y'],
      dtype='object')

In [6]:
add2022 = scout2.append(draft2022)

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\3422529145.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  add2022 = scout2.append(draft2022)


In [7]:
scout = add2022[['player_name_x','pos_abbr_x','weight_x','height_x','school_x','pos_rk_x','ovr_rk_x','grade_x','text1','text2','text3','text4','draft_year','overall','round','team_abbr']]

In [8]:
scout

,player_name_x,pos_abbr_x,weight_x,height_x,school_x,pos_rk_x,ovr_rk_x,grade_x,text1,text2,text3,text4,draft_year,overall,round,team_abbr
0,Bubba Smith,DE,0.0,0.0,Michigan State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,1.0,IND
1,Clinton Jones,RB,0.0,0.0,Michigan State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,2.0,1.0,MIN
2,Steve Spurrier,QB,0.0,0.0,Florida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,3.0,1.0,SF
3,Bob Griese,QB,0.0,0.0,Purdue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,4.0,1.0,MIA
4,George Webster,LB,0.0,0.0,Michigan State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,5.0,1.0,TEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5858,Jeff Gunter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gunter is a versatile edge defender who lines ...,NaN,NaN,NaN,2022.0,NaN,NaN,NaN
5849,Trenton Gill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gill has a strong leg. He led the ACC in punt ...,NaN,NaN,NaN,2022.0,NaN,NaN,NaN
5918,Jesse Luketa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Luketa is a relentless run-defender who has go...,NaN,NaN,NaN,2022.0,NaN,NaN,NaN
5870,Marquis Hayes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hayes is a mauler who plays with good leverage...,NaN,NaN,NaN,2022.0,NaN,NaN,NaN


In [9]:
texts = scout[(scout.text1.notna())|(scout.text2.notna())|(scout.text3.notna())|(scout.text4.notna())]

In [10]:
texts['report'] = None

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\982957188.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts['report'] = None


### Remove Post-Draft Analysis text from scouting report

In [11]:
texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())].apply(lambda x: str(x).split("How he fits")[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\533545784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())].apply(lambda x: str(x).split("How he fits")[0])
C:\Users\capta\AppData\Local\Temp\ipykernel_11432\533545784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())].ap

### Remove "What he brings" header from scouting report

In [12]:
texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())].apply(lambda x: str(x).split("What he brings")[1])

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\3585158994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())].apply(lambda x: str(x).split("What he brings")[1])
C:\Users\capta\AppData\Local\Temp\ipykernel_11432\3585158994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.

In [13]:
texts[(texts.text2.str.contains("<a h",na=False) & texts.text1.isna()) & (texts.draft_year > 2008)]

,player_name_x,pos_abbr_x,weight_x,height_x,school_x,pos_rk_x,ovr_rk_x,grade_x,text1,text2,text3,text4,draft_year,overall,round,team_abbr,report
8561,Matthew Stafford,QB,225.0,74.250,Georgia,1.0,7.0,96.0,NaN,Stafford has very good upside. He has a very s...,NaN,NaN,2009.0,1.0,1.0,DET,None
8565,Mark Sanchez,QB,227.0,74.125,USC,2.0,8.0,95.0,NaN,Sanchez doesn't have as much arm strength or t...,NaN,NaN,2009.0,5.0,1.0,NYJ,None
8570,Michael Crabtree,WR,215.0,73.375,Texas Tech,1.0,3.0,96.0,NaN,Despite leaving school after his redshirt soph...,NaN,NaN,2009.0,10.0,1.0,SF,None
8571,Aaron Maybin,DE,249.0,75.750,Penn State,1.0,6.0,96.0,NaN,Maybin has the quick first step and the athlet...,NaN,NaN,2009.0,11.0,1.0,BUF,None
8572,Knowshon Moreno,RB,217.0,70.625,Georgia,1.0,12.0,94.0,NaN,We place a high value on competitiveness when ...,NaN,NaN,2009.0,12.0,1.0,DEN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9741,Alec Ogletree,ILB,242.0,74.500,Georgia,1.0,19.0,92.0,NaN,: Ogletree is a freakish athlete who on tape l...,NaN,NaN,2013.0,30.0,1.0,STL,None
9742,Travis Frederick,C,312.0,75.625,Wisconsin,1.0,71.0,78.0,NaN,": Frederick is a nasty, versatile interior lin...",NaN,NaN,2013.0,31.0,1.0,DAL,None
9743,Matt Elam,S,208.0,69.875,Florida,2.0,27.0,89.0,NaN,: The biggest knock on Elam is his lack of ide...,NaN,NaN,2013.0,32.0,1.0,BAL,None
10338,Dante Fowler Jr.,DE,261.0,74.625,Florida,2.0,4.0,93.0,NaN,:</strong> Fowler is the best all-around edge ...,NaN,NaN,2015.0,3.0,1.0,JAX,None


In [14]:
texts.loc[(texts.text1.notna()),'report'] = texts.loc[(texts.text1.notna())].text1 

In [15]:
texts.loc[(texts.text4.notna())&(texts.text1.isna()),'report'] = texts.loc[(texts.text4.notna())&(texts.text1.isna())].text4

In [16]:
texts.loc[(texts.report.isna())&(texts.text1.isna())] #= texts.loc[(texts.text4.notna())&(texts.text1.isna())].text2

,player_name_x,pos_abbr_x,weight_x,height_x,school_x,pos_rk_x,ovr_rk_x,grade_x,text1,text2,text3,text4,draft_year,overall,round,team_abbr,report
7406,Logan Mankins,OG,307.0,0.000,Fresno State,NaN,NaN,NaN,NaN,The Patriots lost OG Joe Andruzzi to free agen...,NaN,NaN,2005.0,32.0,1.0,NE,None
8429,Reggie Corner,CB,181.0,69.000,Akron,24.0,NaN,40.0,NaN,Corner has good athletic ability and top-end s...,NaN,NaN,2008.0,114.0,4.0,BUF,None
8450,Josh Sitton,OT,320.0,75.625,UCF,45.0,NaN,20.0,NaN,Sitton has excellent size but lacks ideal expl...,NaN,NaN,2008.0,135.0,4.0,GB,None
8455,Brandon Carr,CB,204.0,72.000,Grand Valley State,54.0,NaN,20.0,NaN,Carr has good size for a corner. He has good a...,NaN,NaN,2008.0,140.0,5.0,KC,None
8469,Kroy Biermann,OLB,246.0,74.875,Montana,32.0,NaN,30.0,NaN,"Biermann played defensive end in college, but ...",NaN,NaN,2008.0,154.0,5.0,ATL,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12468,Joseph Charlton,P,195.0,76.500,South Carolina,2.0,202.0,48.0,NaN,Charlton is a right-footed punter who owns the...,NaN,NaN,2020.0,NaN,NaN,NaN,None
12475,Benito Jones,DT,316.0,73.000,Ole Miss,18.0,219.0,44.0,NaN,"Jones is a three-year starter with good size, ...",NaN,NaN,2020.0,NaN,NaN,NaN,None
12480,Josiah Coatney,DT,308.0,75.375,Ole Miss,19.0,225.0,42.0,NaN,"Coatney has average size, short arms and below...",NaN,NaN,2020.0,NaN,NaN,NaN,None
12520,Rico Dowdle,RB,213.0,71.250,South Carolina,29.0,289.0,32.0,NaN,Dowdle is a well-built back with average speed...,NaN,NaN,2020.0,NaN,NaN,NaN,None


In [17]:
missing = texts[texts.report.isna()]
missing.draft_year

7406     2005.0
8429     2008.0
8450     2008.0
8455     2008.0
8469     2008.0
          ...  
12468    2020.0
12475    2020.0
12480    2020.0
12520    2020.0
12535       NaN
Name: draft_year, Length: 1831, dtype: float64

In [18]:
texts.loc[(texts.text2.notna())&(texts.text1.isna()),'report'] = texts.loc[(texts.text2.notna())&(texts.text1.isna())].text2 

In [19]:
texts.loc[(texts.text2.isna())&(texts.text1.isna())&(texts.text3.notna()),'report'] = texts.loc[(texts.text2.isna())&(texts.text1.isna())&(texts.text3.notna())].text3 

In [20]:
texts.loc[(texts.text2.isna())&(texts.text1.isna())&(texts.text3.notna())]

,player_name_x,pos_abbr_x,weight_x,height_x,school_x,pos_rk_x,ovr_rk_x,grade_x,text1,text2,text3,text4,draft_year,overall,round,team_abbr,report
12279,Bryan Edwards,WR,212.0,74.750,South Carolina,16.0,100.0,69.0,NaN,NaN,Edwards is a bigger receiver with average leng...,The Raiders did well to land Henry Ruggs III i...,2020.0,81.0,3.0,LV,Edwards is a bigger receiver with average leng...
12535,DeAngelo Malone,OLB,231.0,75.125,Western Kentucky,9.0,202.0,50.0,NaN,NaN,"Malone had 99 tackles, 21 tackles for loss and...",NaN,NaN,NaN,NaN,NaN,"Malone had 99 tackles, 21 tackles for loss and..."


In [21]:
#texts.loc[(texts.report.isna()),'report'] = texts.loc[(texts.report.isna())].text1

In [22]:
texts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5047 entries, 7375 to 5967
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   player_name_x  5047 non-null   object 
 1   pos_abbr_x     4824 non-null   object 
 2   weight_x       4824 non-null   float64
 3   height_x       4824 non-null   float64
 4   school_x       4824 non-null   object 
 5   pos_rk_x       4708 non-null   float64
 6   ovr_rk_x       4129 non-null   float64
 7   grade_x        4714 non-null   float64
 8   text1          3215 non-null   object 
 9   text2          3789 non-null   object 
 10  text3          534 non-null    object 
 11  text4          2 non-null      object 
 12  draft_year     5046 non-null   float64
 13  overall        3981 non-null   float64
 14  round          3981 non-null   float64
 15  team_abbr      3981 non-null   object 
 16  report         5047 non-null   object 
dtypes: float64(8), object(9)
memory usage: 838.8+ KB


In [23]:
texts.reset_index(inplace=True)

In [24]:
texts.drop(['index'],axis=1,inplace=True)

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\3926660189.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.drop(['index'],axis=1,inplace=True)


In [25]:
texts.report

0       Nephew of Michigan State head coach John L. Sm...
1       Brown has played second fiddle to Carnell Will...
2       Edwards' father, Stanley, played at Michigan a...
3       Benson was drafted by the Los Angeles Dodgers ...
4       Williams started two games and played in nine ...
                              ...                        
5042    Gunter is a versatile edge defender who lines ...
5043    Gill has a strong leg. He led the ACC in punt ...
5044    Luketa is a relentless run-defender who has go...
5045    Hayes is a mauler who plays with good leverage...
5046    Purdy started 46 games to end his career and o...
Name: report, Length: 5047, dtype: object

In [26]:
texts.report[texts.report.str.contains('<br>')]

30      The Patriots lost OG Joe Andruzzi to free agen...
1332    : There are concerns about the system Bradford...
1333    : Suh lacks elite initial quickness but is sti...
1334    : McCoy has exceptional quickness for a 295-po...
1335    : This a risky pick. There are concerns about ...
                              ...                        
2378    : Shembo needs to get stronger at the point of...
2399    Murray lacks ideal size and physical tools, pa...
2400    McCarron has an average skill set and must cle...
2418    Fales has intriguing tools as a rhythm and tim...
4698    Cheeseman is a consistent snapper who gets goo...
Name: report, Length: 564, dtype: object

In [27]:
texts['rawreport'] = texts.report

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\3885968547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts['rawreport'] = texts.report


In [28]:
texts.report[texts.report.str.contains('<br>')]

30      The Patriots lost OG Joe Andruzzi to free agen...
1332    : There are concerns about the system Bradford...
1333    : Suh lacks elite initial quickness but is sti...
1334    : McCoy has exceptional quickness for a 295-po...
1335    : This a risky pick. There are concerns about ...
                              ...                        
2378    : Shembo needs to get stronger at the point of...
2399    Murray lacks ideal size and physical tools, pa...
2400    McCarron has an average skill set and must cle...
2418    Fales has intriguing tools as a rhythm and tim...
4698    Cheeseman is a consistent snapper who gets goo...
Name: report, Length: 564, dtype: object

In [29]:
texts.report[texts.report.str.contains('<br>')] = texts.report[texts.report.str.contains('<br>')].apply(lambda x: str(x).split('<br>')[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\1299301649.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<br>')] = texts.report[texts.report.str.contains('<br>')].apply(lambda x: str(x).split('<br>')[0])


In [38]:
texts.report[texts.report.str.contains('<p>')] = texts.report[texts.report.str.contains('<p>')].apply(lambda x: str(x).replace('<p>',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\108085310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<p>')] = texts.report[texts.report.str.contains('<p>')].apply(lambda x: str(x).replace('<p>',""))


In [39]:
texts.report[texts.report.str.contains('</p>')] = texts.report[texts.report.str.contains('</p>')].apply(lambda x: str(x).replace('</p>',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\4111044159.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('</p>')] = texts.report[texts.report.str.contains('</p>')].apply(lambda x: str(x).replace('</p>',""))


In [49]:
texts.report[texts.report.str.contains('<a')] = texts.report[texts.report.str.contains('<a')].apply(lambda x: str(x).split('<a')[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\3988211471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<a')] = texts.report[texts.report.str.contains('<a')].apply(lambda x: str(x).split('<a')[0])


In [58]:
texts.report[texts.report.str.contains('Video analysis')] = texts.report[texts.report.str.contains('Video analysis')].apply(lambda x: str(x).split('Video analysis')[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\469107434.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('Video analysis')] = texts.report[texts.report.str.contains('Video analysis')].apply(lambda x: str(x).split('Video analysis')[0])


In [84]:
texts.report[texts.report.str.contains('</br>\r\n')] = texts.report[texts.report.str.contains('</br>\r\n')].apply(lambda x: str(x).replace('</br>\r\n',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\14571374.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('</br>\r\n')] = texts.report[texts.report.str.contains('</br>\r\n')].apply(lambda x: str(x).replace('</br>\r\n',""))


In [94]:
texts.report[texts.report.str.contains('<em>')] = texts.report[texts.report.str.contains('<em>')].apply(lambda x: str(x).split('<em>')[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\613720955.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<em>')] = texts.report[texts.report.str.contains('<em>')].apply(lambda x: str(x).split('<em>')[0])


In [108]:
texts.report[texts.report.str.contains('<i>--')] = texts.report[texts.report.str.contains('<i>--')].apply(lambda x: str(x).split('<i>--')[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\3629875625.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<i>--')] = texts.report[texts.report.str.contains('<i>--')].apply(lambda x: str(x).split('<i>--')[0])


In [115]:
texts.report[texts.report.str.contains('<i> -')] = texts.report[texts.report.str.contains('<i> -')].apply(lambda x: str(x).split('<i> -')[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\3151611948.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<i> -')] = texts.report[texts.report.str.contains('<i> -')].apply(lambda x: str(x).split('<i> -')[0])


In [118]:
texts.report[texts.report.str.contains('<i>')] = texts.report[texts.report.str.contains('<i>')].apply(lambda x: str(x).replace('<i>',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\3312374925.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<i>')] = texts.report[texts.report.str.contains('<i>')].apply(lambda x: str(x).replace('<i>',""))


In [121]:
texts.report[texts.report.str.contains('</i>')] = texts.report[texts.report.str.contains('</i>')].apply(lambda x: str(x).replace('</i>',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\1020409820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('</i>')] = texts.report[texts.report.str.contains('</i>')].apply(lambda x: str(x).replace('</i>',""))


In [127]:
texts.report[texts.report.str.contains('</strong> ')] = texts.report[texts.report.str.contains('</strong> ')].apply(lambda x: str(x).replace('</strong> ',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\261690751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('</strong> ')] = texts.report[texts.report.str.contains('</strong> ')].apply(lambda x: str(x).replace('</strong> ',""))


In [137]:
texts.report[texts.report.str.contains('\r\n<strong>')] = texts.report[texts.report.str.contains('\r\n<strong>')].apply(lambda x: str(x).replace('\r\n<strong>',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\3009428709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('\r\n<strong>')] = texts.report[texts.report.str.contains('\r\n<strong>')].apply(lambda x: str(x).replace('\r\n<strong>',""))


In [140]:
texts.report[texts.report.str.contains('</strong>\r\n')] = texts.report[texts.report.str.contains('</strong>\r\n')].apply(lambda x: str(x).replace('</strong>\r\n',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\1498217247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('</strong>\r\n')] = texts.report[texts.report.str.contains('</strong>\r\n')].apply(lambda x: str(x).replace('</strong>\r\n',""))


In [143]:
texts.report[texts.report.str.contains('<strong>')] = texts.report[texts.report.str.contains('<strong>')].apply(lambda x: str(x).replace('<strong>',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\277396404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<strong>')] = texts.report[texts.report.str.contains('<strong>')].apply(lambda x: str(x).replace('<strong>',""))


In [151]:
texts.report[texts.report.str.contains('</p')] = texts.report[texts.report.str.contains('</p')].apply(lambda x: str(x).replace('</p',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_11432\3646959283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('</p')] = texts.report[texts.report.str.contains('</p')].apply(lambda x: str(x).replace('</p',""))


In [160]:
texts.report[0]

"Nephew of Michigan State head coach John L. Smith.  Alex attempted only five passes as a freshman reserve QB in 2002.  He took over the starting job three games into his sophomore season vs. Cal in 2003 and wound up completing 65-percent of his passes for 2,247 yards with 15 touchdowns compared to just three interceptions.   He also rushed for 452 yards in 2003.  As a junior in 2004, Smith started all 12 games, went undefeated as a starter and led the Utes to a Fiesta Bowl win over Pittsburgh.  Prior to declaring himself available for the 2005 NFL draft, Smith completed 66.1-percent of his passes for 2,624 yards with 28 TD's and four INT's, while also rushing for another 563 yards and 10 TD's.  Smith could have used another year at the collegiate level in order to bulk up and improve his overall strength, but his decision makes sense following a nearly perfect junior season and with head coach Urban Meyer leaving for Florida.  Smith still needs to get bigger and he lacks ideal arm str

In [161]:
texts.columns

Index(['player_name_x', 'pos_abbr_x', 'weight_x', 'height_x', 'school_x',
       'pos_rk_x', 'ovr_rk_x', 'grade_x', 'text1', 'text2', 'text3', 'text4',
       'draft_year', 'overall', 'round', 'team_abbr', 'report', 'rawreport'],
      dtype='object')

In [165]:
report = texts[['player_name_x','report','round','draft_year','overall','pos_abbr_x','weight_x','height_x','pos_rk_x','ovr_rk_x','grade_x','school_x']]

In [166]:
report.columns=['player','report','round','year','pick','pos','weight','height','pos_rk','ovr_rk','grade','school']

In [167]:
report

,player,report,round,year,pick,pos,weight,height,pos_rk,ovr_rk,grade,school
0,Alex Smith,Nephew of Michigan State head coach John L. Sm...,1.0,2005.0,1.0,QB,217.0,76.125,2.0,3.0,98.0,Utah
1,Ronnie Brown,Brown has played second fiddle to Carnell Will...,1.0,2005.0,2.0,RB,233.0,72.250,1.0,4.0,98.0,Auburn
2,Braylon Edwards,"Edwards' father, Stanley, played at Michigan a...",1.0,2005.0,3.0,WR,211.0,74.875,1.0,1.0,99.0,Michigan
3,Cedric Benson,Benson was drafted by the Los Angeles Dodgers ...,1.0,2005.0,4.0,RB,222.0,70.500,3.0,10.0,96.0,Texas
4,Carnell Williams,Williams started two games and played in nine ...,1.0,2005.0,5.0,RB,217.0,70.875,2.0,6.0,97.0,Auburn
...,...,...,...,...,...,...,...,...,...,...,...,...
5042,Jeff Gunter,Gunter is a versatile edge defender who lines ...,NaN,2022.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5043,Trenton Gill,Gill has a strong leg. He led the ACC in punt ...,NaN,2022.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5044,Jesse Luketa,Luketa is a relentless run-defender who has go...,NaN,2022.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5045,Marquis Hayes,Hayes is a mauler who plays with good leverage...,NaN,2022.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
report2022 = report[report.year == 2022]

In [170]:
report2022[['player','report','year']]

,player,report,year
4824,Travon Walker,Walker is an EDGE defender with an excellent c...,2022.0
4825,Aidan Hutchinson,Hutchinson is a notch below elite in terms of ...,2022.0
4826,Derek Stingley,Stingley is a physical press corner with great...,2022.0
4827,Sauce Gardner,Gardner is the best press-man cornerback in th...,2022.0
4828,Kayvon Thibodeaux,Thibodeaux has the explosive first step to get...,2022.0
...,...,...,...
5042,Jeff Gunter,Gunter is a versatile edge defender who lines ...,2022.0
5043,Trenton Gill,Gill has a strong leg. He led the ACC in punt ...,2022.0
5044,Jesse Luketa,Luketa is a relentless run-defender who has go...,2022.0
5045,Marquis Hayes,Hayes is a mauler who plays with good leverage...,2022.0


In [175]:
combine = pd.read_parquet('./Data/origround.parquet')

In [176]:
combine2022 = combine[combine.year == 2022]

In [177]:
combine2022

,player,year,pos,school_name,height,weight,forty_yd,vertical,bench_reps,broad_jump,...,fumbles_forced_season,fumbles_forced_career,team,pick,college_conference,Age,Round,speed,shuttle_agility,cone_agility
5743,Cal Adomitis,2022.0,LS,Pittsburgh,74.0,235.0,4.97,29.5,18.0,107.0,...,NaN,NaN,nan,NaN,None,NaN,8.0,77.032200,103.164424,110.081143
5744,Austin Allen,2022.0,TE,Nebraska,80.0,253.0,4.83,34.0,NaN,121.0,...,NaN,NaN,nan,NaN,None,NaN,8.0,92.974102,153.642767,147.521866
5745,Chase Allen,2022.0,TE,Iowa St.,78.0,251.0,NaN,33.5,NaN,117.0,...,NaN,NaN,nan,NaN,None,NaN,8.0,NaN,130.343330,144.489984
5746,Christopher Allen,2022.0,EDGE,Alabama,76.0,241.0,NaN,NaN,NaN,NaN,...,NaN,3.0,nan,NaN,None,NaN,8.0,NaN,NaN,NaN
5747,Tyler Allgeier,2022.0,RB,BYU,71.0,224.0,4.60,33.0,NaN,120.0,...,NaN,NaN,Atlanta Falcons,151.0,Independent,22.0,5.0,100.056818,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6062,Devonte Wyatt,2022.0,DT,Georgia,75.0,304.0,4.77,29.0,NaN,111.0,...,2.0,2.0,Green Bay Packers,28.0,Southeastern Conference,24.0,1.0,117.443791,NaN,NaN
6063,Jalen Wydermyer,2022.0,TE,Texas A&M,76.0,255.0,NaN,NaN,NaN,NaN,...,NaN,NaN,nan,NaN,None,NaN,8.0,NaN,NaN,NaN
6064,Cade York,2022.0,K,LSU,73.0,206.0,NaN,NaN,12.0,NaN,...,NaN,NaN,Cleveland Browns,124.0,Southeastern Conference,21.0,4.0,NaN,NaN,NaN
6065,Nick Zakelj,2022.0,OL,Fordham,78.0,316.0,5.13,28.5,27.0,110.0,...,NaN,NaN,San Francisco 49ers,187.0,Patriot League,23.0,6.0,91.253098,128.420232,135.772549


In [ ]:
#texts.to_parquet('./Data/draft_reports.parquet')